<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
print(platform.system())
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() in ('Windows','Darwin'):
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

Darwin
El dataset ya se encuentra descargado


In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['prince.txt',
 'dickinson.txt',
 'notorious-big.txt',
 'beatles.txt',
 'bob-dylan.txt',
 'bjork.txt',
 'johnny-cash.txt',
 'disney.txt',
 'janisjoplin.txt',
 'kanye.txt',
 'bob-marley.txt',
 'leonard-cohen.txt',
 'ludacris.txt',
 'adele.txt',
 'alicia-keys.txt',
 'joni-mitchell.txt',
 'amy-winehouse.txt',
 'lorde.txt',
 'rihanna.txt',
 'Kanye_West.txt',
 'nirvana.txt',
 'cake.txt',
 'bieber.txt',
 'notorious_big.txt',
 'missy-elliott.txt',
 'dolly-parton.txt',
 'jimi-hendrix.txt',
 'michael-jackson.txt',
 'al-green.txt',
 'lil-wayne.txt',
 'lady-gaga.txt',
 'lin-manuel-miranda.txt',
 'nursery_rhymes.txt',
 'dj-khaled.txt',
 'radiohead.txt',
 'patti-smith.txt',
 'blink-182.txt',
 'Lil_Wayne.txt',
 'dr-seuss.txt',
 'r-kelly.txt',
 'drake.txt',
 'britney-spears.txt',
 'bruce-springsteen.txt',
 'nicki-minaj.txt',
 'kanye-west.txt',
 'paul-simon.txt',
 'nickelback.txt',
 'eminem.txt',
 'bruno-mars.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

/var/folders/h0/qqbs9kr17cvg14gkx_xt31fm0000gn/T/ipykernel_55737/3849064916.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)


,0
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [6]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar embeddings

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 113153.515625
Loss after epoch 1: 65766.453125
Loss after epoch 2: 65685.21875
Loss after epoch 3: 65358.125
Loss after epoch 4: 63343.09375
Loss after epoch 5: 63227.71875
Loss after epoch 6: 62387.15625
Loss after epoch 7: 61794.28125
Loss after epoch 8: 58445.0
Loss after epoch 9: 55517.0625
Loss after epoch 10: 54163.4375
Loss after epoch 11: 52553.75
Loss after epoch 12: 50621.9375
Loss after epoch 13: 48807.0
Loss after epoch 14: 47483.25
Loss after epoch 15: 46763.125
Loss after epoch 16: 45516.75
Loss after epoch 17: 43179.375
Loss after epoch 18: 40640.875
Loss after epoch 19: 39437.75
Loss after epoch 20: 39499.625
Loss after epoch 21: 38491.0
Loss after epoch 22: 38217.5
Loss after epoch 23: 38213.25
Loss after epoch 24: 37593.5
Loss after epoch 25: 36258.375
Loss after epoch 26: 37001.5
Loss after epoch 27: 35743.125
Loss after epoch 28: 35892.0
Loss after epoch 29: 35381.0
Loss after epoch 30: 35098.125
Loss after epoch 31: 35065.125
Loss after epoch 32

(1570676, 2877400)

### 4 - Ensayar

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('since', 0.5275837182998657),
 ('harm', 0.45388665795326233),
 ('pretty', 0.4391310214996338),
 ('woah', 0.42252862453460693),
 ('send', 0.4049132466316223),
 ('wind', 0.39874762296676636),
 ('right', 0.3969101309776306),
 ('cry', 0.3848605155944824),
 ('loving', 0.3830673396587372),
 ('round', 0.38202229142189026)]

In [15]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('back', -0.012405377812683582),
 ('must', -0.02043626271188259),
 ('seems', -0.03576658293604851),
 ('had', -0.04169529303908348),
 ('the', -0.05981848016381264),
 ('who', -0.06273943930864334),
 ('fool', -0.06314462423324585),
 ('up', -0.06579700857400894),
 ('years', -0.06620099395513535),
 ('words', -0.0739060789346695)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('seven', 0.7449806928634644),
 ('sixty', 0.6629864573478699),
 ('five', 0.5831441283226013),
 ('feed', 0.5518248081207275),
 ('six', 0.5417882204055786),
 ('until', 0.5166842937469482),
 ('eight', 0.47971728444099426),
 ('understand', 0.4728665351867676),
 ('then', 0.4598292410373688),
 ('taxman', 0.45808249711990356)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[('sunshine', 0.5870924592018127),
 ('thing', 0.5788530707359314),
 ('buy', 0.5515603423118591),
 ('than', 0.5387270450592041),
 ('friend', 0.508412778377533)]

In [20]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["submarine"])

[("didn't", -0.05816022679209709),
 ('me', -0.07965023070573807),
 ('your', -0.0935412123799324),
 ('get', -0.11667032539844513),
 ("don't", -0.11831920593976974),
 ('hair', -0.11834351718425751),
 ("you'll", -0.1237618699669838),
 ('but', -0.12485594302415848),
 ('him', -0.1317250281572342),
 ('only', -0.13200761377811432)]

In [21]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.1956341   0.46621168 -0.06660655  0.02165367  0.08101235 -0.3219082
 -0.01463633  0.08634405 -0.18000494  0.23414926  0.07742847  0.02185819
 -0.3555957   0.3780601   0.07576889 -0.45334563 -0.10954668  0.35148415
  0.07809892 -0.01911014  0.12689915 -0.06490382  0.1084874  -0.14517105
 -0.20539439  0.13227452 -0.00707026  0.03213025 -0.23282662 -0.49377096
  0.09179693 -0.1529671   0.14190622 -0.00689983 -0.211404    0.20045
  0.22013322  0.23314881 -0.22769444  0.29296845  0.27795118  0.08189646
  0.04292981  0.05165513  0.3052322   0.22297332  0.02010734 -0.25156057
  0.03521031 -0.37857947 -0.20870335 -0.17146046  0.2341676   0.21831241
  0.09889584  0.22251374  0.3620141   0.08973641  0.3918357  -0.26566458
  0.02307955 -0.297603   -0.46310088  0.08634447  0.13363405 -0.16969357
  0.02589733  0.4634659  -0.06474268 -0.08119601  0.00654426 -0.01480758
  0.24816532 -0.11445356  0.76827675  0.08918052  0.08226225 -0.03231777
 -0.10995325  0.17296755 -0.13001768  0.1913541  -0.111

In [22]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('babe', 0.44351086020469666),
 ('hope', 0.41372379660606384),
 ('new', 0.40814435482025146),
 ('alright', 0.4073256552219391),
 ('sunshine', 0.3966488540172577),
 ('sad', 0.38147827982902527),
 ('real', 0.38124582171440125),
 ('need', 0.3748636543750763),
 ("ain't", 0.36247390508651733)]

In [23]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('babe', 0.44351086020469666),
 ('hope', 0.41372379660606384),
 ('new', 0.40814435482025146),
 ('alright', 0.4073256850242615),
 ('sunshine', 0.3966488540172577),
 ('sad', 0.38147827982902527),
 ('real', 0.38124585151672363),
 ('need', 0.3748636543750763),
 ("ain't", 0.36247390508651733),
 ("somethin'", 0.3618330955505371)]

### 5 - Visualizar agrupación de vectores

In [24]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [28]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

In [29]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab

In [30]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.